In [7]:
import tensorflow as tf
import numpy as np

In [ ]:
sess = tf.InteractiveSession()

In [8]:
# retrieve data sets from data arrays path
Xadj_train = np.load('../data/arrays/Xadj_train.npy')
y_train = np.load('../data/arrays/y_train.npy')

In [9]:
Xadj_train.shape, y_train.shape

((8995, 24, 1), (8995, 168))

In [10]:
# y_train array keeps 168 possible targets, one for each hour in the week
# get only targets for the first model (first-step-ahead)
n = 0
ytarget_train = y_train[:,n]

In [11]:
# required for input_fn's
ytarget_train = ytarget_train.reshape(ytarget_train.shape[0], 1)

In [12]:
Xadj_train.shape, ytarget_train.shape

((8995, 24, 1), (8995, 1))

In [13]:
def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

In [1]:
list_of_values = [lecture[0] for lecture in Xadj_train[0]]
list_of_values

NameError: name 'Xadj_train' is not defined

In [19]:
def _float_feature_from_list_of_values(list_of_values):
  """Returns a float_list from a list of floats / doubles."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_values))

In [20]:
adjacent_hours = _float_feature_from_list_of_values(list_of_values)
adjacent_hours

float_list {
  value: 0.5809716582298279
  value: 0.6275303363800049
  value: 0.6680161952972412
  value: 0.6538461446762085
  value: 0.6761133670806885
  value: 0.7408906817436218
  value: 0.6862348318099976
  value: 0.5769230723381042
  value: 0.5526315569877625
  value: 0.5303643941879272
  value: 0.5870445370674133
  value: 0.5121457576751709
  value: 0.4615384638309479
  value: 0.5566801428794861
  value: 0.6255060434341431
  value: 0.6497975587844849
  value: 0.6356275081634521
  value: 0.6356275081634521
  value: 0.5020242929458618
  value: 0.3785425126552582
  value: 0.36032387614250183
  value: 0.4615384638309479
  value: 0.4777328073978424
  value: 0.5323886871337891
}

In [21]:
_float_feature(ytarget_train[0])

float_list {
  value: 0.6336032152175903
}

In [19]:
# create a TFRecord example for just the first row in the dataset
example = tf.train.Example(
    # features within the example
    features=tf.train.Features(
        # individual feature definition
        feature={'adjacent_hours': _float_feature_from_list_of_values([lecture[0] for lecture in Xadj_train[0]]),
                 'target': _float_feature(ytarget_train[0]),
                }
    )
)

print(example)

features {
  feature {
    key: "adjacent_hours"
    value {
      float_list {
        value: 0.5809716582298279
        value: 0.6275303363800049
        value: 0.6680161952972412
        value: 0.6538461446762085
        value: 0.6761133670806885
        value: 0.7408906817436218
        value: 0.6862348318099976
        value: 0.5769230723381042
        value: 0.5526315569877625
        value: 0.5303643941879272
        value: 0.5870445370674133
        value: 0.5121457576751709
        value: 0.4615384638309479
        value: 0.5566801428794861
        value: 0.6255060434341431
        value: 0.6497975587844849
        value: 0.6356275081634521
        value: 0.6356275081634521
        value: 0.5020242929458618
        value: 0.3785425126552582
        value: 0.36032387614250183
        value: 0.4615384638309479
        value: 0.4777328073978424
        value: 0.5323886871337891
      }
    }
  }
  feature {
    key: "target"
    value {
      float_list {
        value: 0.6336032

In [20]:
example.SerializeToString()

b'\n\x8c\x01\nv\n\x0eadjacent_hours\x12d\x12b\n`\x8f\xba\x14?\xd4\xa5 ?\x1c\x03+?vb\'?\xc4\x15-?\x03\xab=?\x16\xad/?;\xb1\x13?Cy\r?\xf6\xc5\x07?\x8dH\x16?\xfc\x1b\x03?\xc5N\xec>\x97\x82\x0e?*! ?"Y&?|\xb8"?|\xb8"?\xaa\x84\x00?S\xd0\xc1>_|\xb8>\xc5N\xec>e\x99\xf4>\xa0J\x08?\n\x12\n\x06target\x12\x08\x12\x06\n\x04\xd23"?'

In [21]:
# write the basic `tf.Example` (only the first row of adjacent hours training set) to a file
with tf.io.TFRecordWriter('../data/tfrecord/first_row.tfrecord') as writer:
    serialized_example = example.SerializeToString()
    writer.write(serialized_example)

In [24]:
# now read the dataset from TFRecord file using non-deprecated methods from tf.data module
first_row_train_raw_dataset = tf.data.TFRecordDataset('../data/tfrecord/first_row.tfrecord')
first_row_train_raw_dataset

<TFRecordDataset shapes: (), types: tf.string>

In [25]:
first_row_train_raw_dataset.output_types

tf.string

In [26]:
first_row_train_raw_dataset.output_shapes

TensorShape([])

In [27]:
# can I access to the binary, string-based, raw dataset using a one-shot iterator?
iterator = first_row_train_raw_dataset.make_one_shot_iterator()
next_element = iterator.get_next()

# there is only one row in the raw dataset
value = sess.run(next_element)
print(value)

b'\n\x8c\x01\nv\n\x0eadjacent_hours\x12d\x12b\n`\x8f\xba\x14?\xd4\xa5 ?\x1c\x03+?vb\'?\xc4\x15-?\x03\xab=?\x16\xad/?;\xb1\x13?Cy\r?\xf6\xc5\x07?\x8dH\x16?\xfc\x1b\x03?\xc5N\xec>\x97\x82\x0e?*! ?"Y&?|\xb8"?|\xb8"?\xaa\x84\x00?S\xd0\xc1>_|\xb8>\xc5N\xec>e\x99\xf4>\xa0J\x08?\n\x12\n\x06target\x12\x08\x12\x06\n\x04\xd23"?'


In [48]:
# define features
read_features = {
    'adjacent_hours': tf.VarLenFeature(dtype=tf.float32),
    'target': tf.FixedLenFeature([], dtype=tf.float32)}

In [33]:
def _parse_dataset_function(example_proto):
  # parse the input tf.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, read_features)

parsed_dataset = first_row_train_raw_dataset.map(_parse_dataset_function)
parsed_dataset

<MapDataset shapes: {adjacent_hours: (?,), target: ()}, types: {adjacent_hours: tf.float32, target: tf.float32}>

In [34]:
iterator = parsed_dataset.make_one_shot_iterator()
next_element = iterator.get_next()

# there is only one row in the raw dataset
value = sess.run(next_element)
print(value['adjacent_hours'].values, value['adjacent_hours'].values.shape)

[0.58097166 0.62753034 0.6680162  0.65384614 0.67611337 0.7408907
 0.68623483 0.5769231  0.55263156 0.5303644  0.58704454 0.51214576
 0.46153846 0.55668014 0.62550604 0.64979756 0.6356275  0.6356275
 0.5020243  0.3785425  0.36032388 0.46153846 0.4777328  0.5323887 ] (24,)


In [35]:
# TEST WITH DIFFERENT PARSING FUNCTIONS
def _parse_dataset_function(example_proto):
    # parse the input tf.Example proto using the dictionary above
    row = tf.io.parse_single_example(example_proto, read_features)
    adjacent_hours = tf.reshape(row['adjacent_hours'].values, [-1, 1])
    target = tf.reshape(row['target'], [1,])
    
    return ({'adjacent_hours': adjacent_hours}, target)
    

parsed_dataset = first_row_train_raw_dataset.map(_parse_dataset_function)
parsed_dataset

<MapDataset shapes: ({adjacent_hours: (?, 1)}, (1,)), types: ({adjacent_hours: tf.float32}, tf.float32)>

In [36]:
iterator = parsed_dataset.make_one_shot_iterator()
next_element = iterator.get_next()

# there is only one row in the raw dataset
value = sess.run(next_element)
print(value)

({'adjacent_hours': array([[0.58097166],
       [0.62753034],
       [0.6680162 ],
       [0.65384614],
       [0.67611337],
       [0.7408907 ],
       [0.68623483],
       [0.5769231 ],
       [0.55263156],
       [0.5303644 ],
       [0.58704454],
       [0.51214576],
       [0.46153846],
       [0.55668014],
       [0.62550604],
       [0.64979756],
       [0.6356275 ],
       [0.6356275 ],
       [0.5020243 ],
       [0.3785425 ],
       [0.36032388],
       [0.46153846],
       [0.4777328 ],
       [0.5323887 ]], dtype=float32)}, array([0.6336032], dtype=float32))


In [37]:
# everything works fine, now build the dataset to serialize with more than one record

In [41]:
with tf.io.TFRecordWriter('../data/tfrecord/ten_rows.tfrecord') as writer:
    # get an iterable with the indexes of the NumPy array to be stored in the TFRecord file
    # test with only 10 rows
    for row in np.arange(10):
        example = tf.train.Example(
            # features within the example
            features=tf.train.Features(
                # individual feature definition
                feature={'adjacent_hours': _float_feature_from_list_of_values([lecture[0] for lecture in Xadj_train[row]]),
                         'target': _float_feature(ytarget_train[row]),
                        }
            )
        )
        serialized_example = example.SerializeToString()
        writer.write(serialized_example)

In [43]:
# read and parse the bigger dataset from the bigger TFRecord file

In [42]:
bigger_raw_dataset = tf.data.TFRecordDataset('../data/tfrecord/ten_rows.tfrecord')

In [44]:
bigger_parsed_dataset = bigger_raw_dataset.map(_parse_dataset_function)
parsed_dataset

<MapDataset shapes: ({adjacent_hours: (?, 1)}, (1,)), types: ({adjacent_hours: tf.float32}, tf.float32)>

In [47]:
iterator = bigger_parsed_dataset.make_one_shot_iterator()

for row in np.arange(10):
    next_element = iterator.get_next()
    value = sess.run(next_element)
    print(value)

({'adjacent_hours': array([[0.58097166],
       [0.62753034],
       [0.6680162 ],
       [0.65384614],
       [0.67611337],
       [0.7408907 ],
       [0.68623483],
       [0.5769231 ],
       [0.55263156],
       [0.5303644 ],
       [0.58704454],
       [0.51214576],
       [0.46153846],
       [0.55668014],
       [0.62550604],
       [0.64979756],
       [0.6356275 ],
       [0.6356275 ],
       [0.5020243 ],
       [0.3785425 ],
       [0.36032388],
       [0.46153846],
       [0.4777328 ],
       [0.5323887 ]], dtype=float32)}, array([0.6336032], dtype=float32))
({'adjacent_hours': array([[0.62753034],
       [0.6680162 ],
       [0.65384614],
       [0.67611337],
       [0.7408907 ],
       [0.68623483],
       [0.5769231 ],
       [0.55263156],
       [0.5303644 ],
       [0.58704454],
       [0.51214576],
       [0.46153846],
       [0.55668014],
       [0.62550604],
       [0.64979756],
       [0.6356275 ],
       [0.6356275 ],
       [0.5020243 ],
       [0.3785425 ],
    

In [ ]:
# everything seems to be working fine, now feed a simple Estimator model with
# only the adjacent hours SLDB and test the whole process... tomorrow

In [7]:
# ToDo: verify if targets can be declared as a Feature, as with the feature vector
# later, now just run with the previous, working code
# def train_input_fn():
#     dataset = (
#         tf.data.Dataset.from_tensor_slices(
#             (
#                 {
#                     'adjacent_hours': tf.cast(Xadj_train, tf.float32),
#                     'target': tf.cast(ytarget_train, tf.float32) 
#                 }
#             )
#         )
#     )
#     dataset = dataset.shuffle(buffer_size=9000).repeat(count=1).batch(batch_size=32)    
#     return dataset

In [8]:
def train_input_fn():
    dataset = (
        tf.data.Dataset.from_tensor_slices(
            (
                {
                    'adjacent_hours': tf.cast(Xadj_train, tf.float32)
                },
                tf.cast(ytarget_train, tf.float32)
            )
        )
    )
    dataset = dataset.shuffle(buffer_size=9000).repeat(count=1).batch(batch_size=32)    
    return dataset